In [2]:
import numpy as np
import pandas as pd

In [3]:
#Reading from csv file
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

print(data_train.shape)
print(data_test.shape)

(1460, 81)
(1459, 80)


In [5]:
#train data set
y_train = data_train['SalePrice']
# print(y_train.shape)
X_train = data_train.drop(['Id','SalePrice'], axis = 1) \
                    .select_dtypes(exclude=['object'])

#test data set
X_test = data_test.drop(['Id'], axis = 1) \
                  .select_dtypes(exclude=['object'])

print(X_train.shape)
print(X_test.shape)
print(X_train)

(1460, 36)
(1459, 36)
      MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0             60         65.0     8450            7            5       2003   
1             20         80.0     9600            6            8       1976   
2             60         68.0    11250            7            5       2001   
3             70         60.0     9550            7            5       1915   
4             60         84.0    14260            8            5       2000   
...          ...          ...      ...          ...          ...        ...   
1455          60         62.0     7917            6            5       1999   
1456          20         85.0    13175            6            6       1978   
1457          70         66.0     9042            7            9       1941   
1458          20         68.0     9717            5            6       1950   
1459          20         75.0     9937            5            6       1965   

      YearRemodAdd  MasVnrAre

In [7]:
nan_counts = pd.DataFrame({
    'feature': X_train.columns,
    'n_missing': np.isnan(X_train.values).sum(axis=0)
})
print(nan_counts[nan_counts.n_missing > 0])

# 2) Impute (e.g. with median)
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
X_filled = pd.DataFrame(
    imputer.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

        feature  n_missing
1   LotFrontage        259
7    MasVnrArea          8
24  GarageYrBlt         81


In [10]:
from sklearn.preprocessing import StandardScaler
scaler   = StandardScaler()
X_scaled = scaler.fit_transform(X_filled.values)
m, n     = X_scaled.shape
X_b      = np.c_[np.ones(m), X_scaled]
y_vec    = y_train.values.reshape(-1,1)

# 2) Init
theta    = np.zeros((n+1,1))
alpha    = 1e-3        # ← smaller!
max_iters= 10000

# 3) GD loop (no early stopping for now)
for i in range(max_iters):
    preds   = X_b.dot(theta)
    errors  = preds - y_vec
    grads   = (2/m) * X_b.T.dot(errors)
    theta  -= alpha * grads

# 4) Inspect
print("Intercept:", theta[0,0])
print("First 5 coefs:", theta[1:6,0])

Intercept: 180921.19552489853
First 5 coefs: [-7623.11170737 -1180.86976074  4283.47422569 24046.64754392
  5075.12438205]
